# Dataset 
> Functions for dataset

In [ ]:
# default_exp dataset
# default_cls_lvl 2

## Dataset Basics

In this notebook we are going to define the dataset for our Horse Poo Detector. I do the labeling with [prodi.gy](https://prodi.gy/). From there the data is exported as JSONL [JSON-lines](http://jsonlines.org/). The picture itself is encoded in a base64-URL format. The easiest way is to extract the pictures to a folder and then use the libraries given.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import json
import nbdev
import re
import base64
import PIL
import pathlib
from pathlib import Path
import torch
import jsonlines
from torch.utils.data import Dataset, DataLoader
from torch import nn
from io import BytesIO
from fastai.vision import *

In [ ]:
#export
def read_jsonl(file_path):
    """returns a generator which returns each jsonl line.
    """
    with pathlib.Path(file_path).open('r', encoding='utf8') as f:
        for line in f:
            try:  # hack to handle broken jsonl
                str_json = json.loads(line)
                if str_json['answer'] == 'accept':
                    yield str_json
            except ValueError:
                continue

In [ ]:
rj = read_jsonl('test_data/sample.jsonl')
data_point = next(rj)
assert 'image' in data_point, 'image key expected'
assert 'width' in data_point, 'width key expected'
assert 'height' in data_point, 'height key expected'
assert 'spans' in data_point, 'spans key expected'

In [ ]:
#export
def base64ToImage(base64_string):
    """ convert base64 string to image"""
    base64_string = base64_string[23:]
    imgdata = base64.b64decode(str(base64_string))
    image = PIL.Image.open(io.BytesIO(imgdata))
    #image = np.rollaxis(np.array(image), 2, 0)
    #image = pil2tensor(image,np.float32)
    #image.div_(255)
    #image = Image(image)
    return image

In [ ]:
#export
def add_padding(image:PIL.Image, target_size=640):
    """this function adds padding so we have a rectangulare shape"""
    delta_w = target_size - image.size[0]
    delta_h = target_size - image.size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    new_im = PIL.ImageOps.expand(image, padding)
    return new_im

In [ ]:
rj = read_jsonl('test_data/sample.jsonl')
data_point = next(rj)
image = base64ToImage(data_point['image'])

before = image.size
image = add_padding(image)
after = image.size
assert before != after
assert after[0] == after[1]
assert before[0] == after[0]

In [ ]:
rj = read_jsonl('test_data/sample.jsonl')
data_point = next(rj)
image = base64ToImage(data_point['image'])
assert type(image) == PIL.JpegImagePlugin.JpegImageFile
assert image.size == (640, 352)

In [ ]:
#export 
def span_to_box(jsonl:str, adjust_for_padding=0):
    """converts span data to list with points and labels"""
    box_points = []
    labels = []
    for box in jsonl['spans']:
        x1 = np.array(box['points'])[:,1].min() + adjust_for_padding
        x2 = np.array(box['points'])[:,1].max() + adjust_for_padding
        y1 = np.array(box['points'])[:,0].min()
        y2 = np.array(box['points'])[:,0].max()
        box_points.append([x1,y1,x2,y2])
        labels.append(box['label'])
    return [box_points, labels]

In [ ]:
rj = read_jsonl('test_data/sample.jsonl')
data_point = next(rj)
boxes = span_to_box(data_point)

assert len(boxes) == 2
assert len(boxes[0][0]) == 4
assert len(boxes[1]) == 1


boxes_with_padding = span_to_box(data_point, adjust_for_padding=100)
assert boxes[0][0][0] + 100 == boxes_with_padding[0][0][0]
assert boxes[0][0][2] + 100 == boxes_with_padding[0][0][2]



In [ ]:
#export
def jsonl_convert(path:Path, add_pad=True):
    """convert jsonl to folder  with image files for better handling"""
    if type(path) == str:
        path = Path(path)
    path_target_fld = (path.parent / path.stem)
    path_target_fld.mkdir(parents=True, exist_ok=True)
    boxes = dict()
    for jsonl in read_jsonl(str(path)):
        base64_string = jsonl['image']
        base64_string = base64_string[23:]
        imgdata = base64.b64decode(str(base64_string))
        image = PIL.Image.open(io.BytesIO(imgdata))
        image_size = image.size
        if add_pad:
            image = add_padding(image)
        image.save(str(path_target_fld / jsonl['meta']['file']))

        adjustment = (image_size[0] - image_size[1])/2
        boxes[jsonl['meta']['file']] = span_to_box(jsonl, adjust_for_padding=adjustment)

    with open(path_target_fld / 'boxes.json', 'w') as fp:
        json.dump(boxes, fp)

    return boxes

In [ ]:
#hide 

# delete folder and content
import os, shutil


folder = 'test_data/sample/'
if os.path.isdir(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

    shutil.rmtree(folder)
   

In [ ]:
boxes = jsonl_convert('test_data/sample.jsonl')

assert os.path.isfile(str(Path(folder) / '20190607060607_784617.jpg'))
assert len(os.listdir(folder)) == 5


In [ ]:
boxes

In [ ]:
#export 
def get_labels_for_folder(folder:Path):
    """reads labels from boxes.json file"""
    if type(folder) is str:
        folder = Path(folder)
    with open(str(folder / 'boxes.json'), 'r') as fp:
        data = json.load(fp)
    return data

In [ ]:
folder = Path('test_data/sample')
file = Path('test_data/sample.jsonl')
jsonl_convert(file)
boxes = get_labels_for_folder(folder)

assert len(boxes) == 4
assert '20190607060607_784617.jpg' in boxes.keys()

In [ ]:
folder = 'test_data/sample/' 
boxes = get_labels_for_folder(folder)
transforms = [[flip_lr(p=0.5), brightness(change=(0.4, 0.6), p=0.75), contrast(scale=(0.8, 1.25), p=.75)],[]]

In [ ]:
data = (ObjectItemList
        .from_folder(folder)
        .split_by_rand_pct(0.5)
        .label_from_func(lambda o: boxes[o.name])
        .transform(transforms, tfm_y=True)
        .databunch(bs=2, collate_fn=bb_pad_collate)
       # .normalize(imagenet_stats)
       )

data.show_batch()

## folder operations

In [ ]:
#export 
def pad_images_in_folder(path:Path, num_start=None, num_of_files=None):
    """all immages in the folder will be overwritten with images with padding"""
    iterator = path.iterdir()
    if num_start is None:
        num_start = 0
    
    if num_of_files is None:
        num_of_files = len([f for f in os.listdir(str(path)) if os.path.isfile(os.path.join(path, f))])
        num_of_files -= num_start
        
    stop = num_start + num_of_files
    
    for i in progress_bar(range(num_of_files)):
        if num_start is not None:
            if i < num_start:
                next(iterator)
                continue
            if i > stop:
                break                
        
                
        path_img = next(iterator)
        if path_img.suffix == '.jpg':
            print(path_img)
            img = PIL.Image.open(str(path_img))            
            img = add_padding(img, target_size=640)              
            img.save(str(path_img))

In [ ]:
# create sample data
from shutil import copyfile, rmtree



path_dest = Path('test_data/pad_samples/padded')
path_dest.mkdir(parents=True, exist_ok=True)

path_source = Path('test_data/pad_samples')

for path_src in path_source.iterdir():
    if path_src.suffix == '.jpg':
        print(f'copy from {path_src} to {path_dest / path_src.name}')
        copyfile(path_src, path_dest / path_src.name)

pad_images_in_folder(path_dest, num_start=None, num_of_files=None)


# run test
for path in path_dest.iterdir():
    if path.suffix == '.jpg':        
        image = PIL.Image.open(path)
        image = np.asarray(image)
        assert image.shape == (640,640,3), print(f'file {path} not padded with shape {image.shape} ')   
        
        
# clean up 
import os, shutil
folder = str(path_dest)
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


## Export

In [ ]:
from nbdev.export import *
notebook2script()
